In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---

# Mouse

## Montoro_10x

In [ ]:
montoro_10x = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Montoro_10x/data.h5")
utils.peek(montoro_10x, "build/trachea/Montoro_10x")
montoro_10x.obs.head()

In [ ]:
montoro_10x.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor", rmbatch_module="RMBatch")
montoro_10x_model = cb.directi.fit_DIRECTi(
    montoro_10x, montoro_10x.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
montoro_10x.latent = montoro_10x_model.inference(montoro_10x)

In [ ]:
ax = montoro_10x.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Montoro_10x/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = montoro_10x.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Montoro_10x/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = montoro_10x.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Montoro_10x/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
montoro_10x.write_dataset("build/trachea/Montoro_10x/Montoro_10x.h5")

In [ ]:
%%capture capio
montoro_10x_models = [montoro_10x_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    montoro_10x_models.append(cb.directi.fit_DIRECTi(
        montoro_10x, montoro_10x.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
montoro_10x_blast = cb.blast.BLAST(montoro_10x_models, montoro_10x)
montoro_10x_blast.save("build/trachea/Montoro_10x")

In [ ]:
with open("build/trachea/Montoro_10x/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/trachea/Montoro_10x/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(montoro_10x_blast, "build/trachea/Montoro_10x")

In [ ]:
%%writefile build/trachea/Montoro_10x/predictable.txt
cell_ontology_class
cell_type1

## Plasschaert

In [ ]:
plasschaert = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Plasschaert/data.h5")
utils.peek(plasschaert, "build/trachea/Plasschaert")
plasschaert.obs.head()

In [ ]:
plasschaert.obs.dtypes

In [ ]:
plasschaert_model = cb.directi.fit_DIRECTi(
    plasschaert, plasschaert.uns["seurat_genes"],
    **fixed_model_kwargs
)
plasschaert.latent = plasschaert_model.inference(plasschaert)

In [ ]:
ax = plasschaert.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Plasschaert/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = plasschaert.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Plasschaert/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = plasschaert.visualize_latent("cell_type2", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Plasschaert/cell_type2.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = plasschaert.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Plasschaert/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
plasschaert.write_dataset("build/trachea/Plasschaert/Plasschaert.h5")

In [ ]:
%%capture capio
plasschaert_models = [plasschaert_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    plasschaert_models.append(cb.directi.fit_DIRECTi(
        plasschaert, plasschaert.uns["seurat_genes"],
        **fixed_model_kwargs,
        random_seed=i
    ))
plasschaert_blast = cb.blast.BLAST(plasschaert_models, plasschaert)
plasschaert_blast.save("build/trachea/Plasschaert")

In [ ]:
with open("build/trachea/Plasschaert/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/trachea/Plasschaert/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(plasschaert_blast, "build/trachea/Plasschaert")

In [ ]:
%%writefile build/trachea/Plasschaert/predictable.txt
cell_ontology_class
cell_type1
cell_type2

## Mouse aligned

In [ ]:
if not os.path.exists("build/trachea/ALIGNED_Mus_musculus_Trachea"):
    os.makedirs("build/trachea/ALIGNED_Mus_musculus_Trachea")
montoro_smart_seq2 = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Montoro_Smart-seq2/data.h5")
montoro_10x.obs["montoro_10x_donor"] = montoro_10x.obs["donor"]
mouse = cb.data.ExprDataSet.merge_datasets(dict(
    montoro_10x=montoro_10x, 
    plasschaert=plasschaert,
    montoro_smart_seq2=montoro_smart_seq2
), merge_uns_slots=["seurat_genes"])

In [ ]:
mouse.obs.dtypes

In [ ]:
opt_model_kwargs = dict(
    batch_effect=["dataset_name", "montoro_10x_donor"],
    rmbatch_module=["Adversarial", "RMBatch"],
    rmbatch_module_kwargs=[dict(lambda_reg=0.005), dict()]
)
mouse_model = cb.directi.fit_DIRECTi(
    mouse, mouse.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
mouse.latent = mouse_model.inference(mouse)

In [ ]:
ax = mouse.visualize_latent("cell_ontology_class", width=10, height=10, size=5, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/ALIGNED_Mus_musculus_Trachea/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("dataset_name", width=10, height=10, size=5, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/ALIGNED_Mus_musculus_Trachea/dataset_name.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
mouse.write_dataset("build/trachea/ALIGNED_Mus_musculus_Trachea/ALIGNED_Mus_musculus_Trachea.h5")

In [ ]:
%%capture capio
mouse_models = [mouse_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    mouse_models.append(cb.directi.fit_DIRECTi(
        mouse, mouse.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
mouse_blast = cb.blast.BLAST(mouse_models, mouse)
mouse_blast.save("build/trachea/ALIGNED_Mus_musculus_Trachea")

In [ ]:
with open("build/trachea/ALIGNED_Mus_musculus_Trachea/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/trachea/ALIGNED_Mus_musculus_Trachea/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(mouse_blast, "build/trachea/ALIGNED_Mus_musculus_Trachea")

In [ ]:
%%writefile build/trachea/ALIGNED_Mus_musculus_Trachea/predictable.txt
cell_ontology_class

## Quake_10x_Trachea

In [ ]:
quake_10x_trachea = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_10x_Trachea/data.h5")
utils.peek(quake_10x_trachea, "build/trachea/Quake_10x_Trachea")
quake_10x_trachea.obs.head()

In [ ]:
quake_10x_trachea.obs["cluster"] = "cluster_" + quake_10x_trachea.obs["cluster"].astype(int).astype(str)
quake_10x_trachea.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
quake_10x_trachea_model = cb.directi.fit_DIRECTi(
    quake_10x_trachea, quake_10x_trachea.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
quake_10x_trachea.latent = quake_10x_trachea_model.inference(quake_10x_trachea)

In [ ]:
ax = quake_10x_trachea.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_10x_Trachea/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_trachea.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_10x_Trachea/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_trachea.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_10x_Trachea/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_10x_trachea.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_10x_Trachea/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
quake_10x_trachea.write_dataset("build/trachea/Quake_10x_Trachea/Quake_10x_Trachea.h5")

In [ ]:
%%capture capio
quake_10x_trachea_models = [quake_10x_trachea_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    quake_10x_trachea_models.append(cb.directi.fit_DIRECTi(
        quake_10x_trachea, quake_10x_trachea.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
quake_10x_trachea_blast = cb.blast.BLAST(quake_10x_trachea_models, quake_10x_trachea)
quake_10x_trachea_blast.save("build/trachea/Quake_10x_Trachea")

In [ ]:
with open("build/trachea/Quake_10x_Trachea/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/trachea/Quake_10x_Trachea/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(quake_10x_trachea_blast, "build/trachea/Quake_10x_Trachea")

In [ ]:
%%writefile build/trachea/Quake_10x_Trachea/predictable.txt
cell_ontology_class
cell_type1
cluster

## Quake_Smart-seq2_Trachea

In [ ]:
quake_smart_seq2_trachea = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_Smart-seq2_Trachea/data.h5")
utils.peek(quake_smart_seq2_trachea, "build/trachea/Quake_Smart-seq2_Trachea")
quake_smart_seq2_trachea.obs.head()

In [ ]:
quake_smart_seq2_trachea.obs["cluster"] = "cluster_" + quake_smart_seq2_trachea.obs["cluster"].astype(int).astype(str)
quake_smart_seq2_trachea.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
quake_smart_seq2_trachea_model = cb.directi.fit_DIRECTi(
    quake_smart_seq2_trachea, quake_smart_seq2_trachea.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
quake_smart_seq2_trachea.latent = quake_smart_seq2_trachea_model.inference(quake_smart_seq2_trachea)

In [ ]:
ax = quake_smart_seq2_trachea.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_Smart-seq2_Trachea/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_trachea.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_Smart-seq2_Trachea/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_trachea.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_Smart-seq2_Trachea/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_trachea.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/trachea/Quake_Smart-seq2_Trachea/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
quake_smart_seq2_trachea.write_dataset("build/trachea/Quake_Smart-seq2_Trachea/Quake_Smart-seq2_Trachea.h5")

In [ ]:
%%capture capio
quake_smart_seq2_trachea_models = [quake_smart_seq2_trachea_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    quake_smart_seq2_trachea_models.append(cb.directi.fit_DIRECTi(
        quake_smart_seq2_trachea, quake_smart_seq2_trachea.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
quake_smart_seq2_trachea_blast = cb.blast.BLAST(quake_smart_seq2_trachea_models, quake_smart_seq2_trachea)
quake_smart_seq2_trachea_blast.save("build/trachea/Quake_Smart-seq2_Trachea")

In [ ]:
with open("build/trachea/Quake_Smart-seq2_Trachea/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/trachea/Quake_Smart-seq2_Trachea/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(quake_smart_seq2_trachea_blast, "build/trachea/Quake_Smart-seq2_Trachea")

In [ ]:
%%writefile build/trachea/Quake_Smart-seq2_Trachea/predictable.txt
cell_ontology_class
cell_type1
cluster